<a href="https://colab.research.google.com/github/m1tavares/m1tavares/blob/master/GroupProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  #importing packages
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import norm
 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
 
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure
 
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
 
#import data
news = pd.read_csv('/content/online_news_popularity.csv')
shop = pd.read_csv('/content/online_shoppers_intention.csv')
 
 
#print coloumn headers and the first 5 rows to see data frame.
print(news.columns.values)
print(shop.columns.values)
shop.head()
 
#creating bins for each share ranges
binx = [0,500,1000,1500,2000,2500,3000, 5000,10000,15000,20000, 500000]
#converting "shares" from continious to categorical
pd.cut(news[' shares'], bins=binx).value_counts(sort=False)
news['shares_group'] = pd.cut(news[' shares'], bins=binx)
#converting categorical back into int to be used in our learning methods
news['shares_group'] = pd.factorize(news['shares_group'])[0] + 1
 
news.head(10)

In [ ]:
#checking for missing data points. None were found... data exploration
for col in news.columns:
  pct_missing = np.mean(news[col].isnull())
  print('{} - {}%'.format(col, round(pct_missing*100)))

In [ ]:
#test/train

from pandas.plotting import scatter_matrix
from matplotlib import cm
#all features
feature_names= [' n_tokens_title', ' n_tokens_content',
 ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',
 ' num_hrefs',' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos',
 ' average_token_length', ' num_keywords', ' data_channel_is_lifestyle',
 ' data_channel_is_entertainment', ' data_channel_is_bus',
 ' data_channel_is_socmed', ' data_channel_is_tech',
 ' data_channel_is_world' ,' kw_min_min' ,' kw_max_min', ' kw_avg_min',
 ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg', ' kw_max_avg',
 ' kw_avg_avg', ' self_reference_min_shares', ' self_reference_max_shares',
 ' self_reference_avg_sharess', ' weekday_is_monday', ' weekday_is_tuesday',
 ' weekday_is_wednesday', ' weekday_is_thursday', ' weekday_is_friday',
 ' weekday_is_saturday' ,' weekday_is_sunday' ,' is_weekend' ,' LDA_00',
 ' LDA_01', ' LDA_02', ' LDA_03', ' LDA_04', ' global_subjectivity',
 ' global_sentiment_polarity', ' global_rate_positive_words',
 ' global_rate_negative_words', ' rate_positive_words',
 ' rate_negative_words', ' avg_positive_polarity', ' min_positive_polarity',
 ' max_positive_polarity', ' avg_negative_polarity',
 ' min_negative_polarity', ' max_negative_polarity' ,' title_subjectivity',
 ' title_sentiment_polarity', ' abs_title_subjectivity',
 ' abs_title_sentiment_polarity']
x = news[feature_names]
y = news['shares_group']



#selected few features.
shop_feature_names=['BounceRates', 'ExitRates', 'PageValues']
x1 = shop[shop_feature_names]
y1 = shop['Revenue']

newsx_train, newsx_test, newsy_train, newsy_test = train_test_split(x,y, test_size=0.3, random_state=0)
shopx_train, shopx_test, shopy_train, shopy_test = train_test_split(x1,y1, test_size=0.3, random_state=0)

#scaling data set
scaler = MinMaxScaler(feature_range=(-1,1)).fit(newsx_train)
newsx_train = scaler.fit_transform(newsx_train)
newsx_test = scaler.transform(newsx_test)

shopx_train = scaler.fit_transform(shopx_train)
shopx_test = scaler.transform(shopx_test)

In [ ]:
#SVM
svmnews=SVC()
svmshop=SVC()

svmshop.fit(shopx_train, shopy_train)
print('Accuracy of SVM for shop train data {:.2f}'.format(svmshop.score(shopx_train, shopy_train)))
print('Accuracy of SVM for shop test data {:.2f}'.format(svmshop.score(shopx_test, shopy_test)))

svmnews.fit(newsx_train, newsy_train)

#long run time, low accuracy. 
print('Accuracy of SVM for news train data {:.2f}'.format(svmnews.score(newsx_train, newsy_train)))
print('Accuracy of SVM for news test data {:.2f}'.format(svmnews.score(newsx_test, newsy_test)))



In [ ]:
#knn
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(newsx_train, newsy_train)
print('Accuracy of K-NN for news train data: {:.2f} '.format(knn.score(newsx_train, newsy_train)))
print('Accuracy of K-NN for news test data: {:.2f} '.format(knn.score(newsx_test, newsy_test)))

knn.fit(shopx_train, shopy_train)
print('Accuracy of K-NN for shop train data: {:.2f} '.format(knn.score(shopx_train, shopy_train)))
print('Accuracy of K-NN for shop test data: {:.2f} '.format(knn.score(shopx_test, shopy_test)))


In [ ]:
#decision
clfnews = DecisionTreeClassifier().fit(newsx_train, newsy_train)
clfshop = DecisionTreeClassifier().fit(shopx_train, shopy_train)


#news data
print('Accuracy of Decision Tree classifier on news training set: {:.2f}'
     .format(clfnews.score(newsx_train, newsy_train)))
print('Accuracy of Decision Tree classifier on news test set: {:.2f}'
     .format(clfnews.score(newsx_test, newsy_test)))

#shop data
print('Accuracy of Decision Tree classifier on shop training set: {:.2f}'
     .format(clfshop.score(shopx_train, shopy_train)))
print('Accuracy of Decision Tree classifier on shop test set: {:.2f}'
     .format(clfshop.score(shopx_test, shopy_test)))



In [ ]:
#confusion matrix K-NN for shop
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

pred = knn.predict(shopx_test)
print(confusion_matrix(shopy_test, pred))
print(classification_report(shopy_test, pred))

pred1 = knn.predict(shopx_train)
print(confusion_matrix(shopy_train, pred1))
print(classification_report(shopy_train, pred1))

#confusion matrix K-NN for news
#refitting model to news...
knn.fit(newsx_train, newsy_train)

predx = knn.predict(newsx_test)
print(confusion_matrix(newsy_test, predx))
print(classification_report(newsy_test, predx)) 

predy = knn.predict(newsx_train)
print(confusion_matrix(newsy_train, predy))
print(classification_report(newsy_train, predy))


In [ ]:
pred2 = svmshop.predict(shopx_test)
print(confusion_matrix(shopy_test, pred2))
print(classification_report(shopy_test, pred2))

pred3 = svmshop.predict(shopx_train)
print(confusion_matrix(shopy_train, pred3))
print(classification_report(shopy_train, pred3))


pred4 = svmnews.predict(newsx_test)
print(confusion_matrix(newsy_test, pred4))
print(classification_report(newsy_test, pred4)) 

pred5 = svmnews.predict(newsx_train)
print(confusion_matrix(newsy_train, pred5))
print(classification_report(newsy_train, pred5))

In [ ]:



pred6 = clfnews.predict(newsx_test)
print(confusion_matrix(newsy_test, pred6))
print(classification_report(newsy_test, pred6)) 

pred7 = clfnews.predict(newsx_train)
print(confusion_matrix(newsy_train, pred7))
print(classification_report(newsy_train, pred7))

pred8 = clfshop.predict(shopx_test)
print(confusion_matrix(shopy_test, pred8))
print(classification_report(shopy_test, pred8))

pred9 = clfshop.predict(shopx_train)
print(confusion_matrix(shopy_train, pred9))
print(classification_report(shopy_train, pred9))